In [1]:
#dataloader
from __future__ import print_function, division
import sys
import os
import torch
import numpy as np
import random
import csv

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import Sampler

from pycocotools.coco import COCO

import skimage.io
import skimage.transform
import skimage.color
import skimage

from PIL import Image

In [3]:
# path='C:/Users/user/Desktop/all_img'

In [22]:
#coco=COCO(os.path.join(path+'/'+'train'+'/'+'annotations'+'/'+'instances_'+'train2017'+'.json'))
#class_1=coco.getCatIds()
#coco.loadCats(3)[0]['name']

'AB_BI'

In [2]:
classes=['Larva','disease'] 

In [3]:
#coco형식의 데이터 셋 불러오기
class CocoDataset(Dataset):
    def __init__(self,data_dir,set_name,transform=None): #source code에는 set_name=train2017로 고정되어있음
        self.data_dir=data_dir #train,valid, test까지의 경로,O
        self.set_name=set_name #train2017, val2017->train까지의 경로일 시, test경로일시->test2017,O
        self.transform=transform #이미지 정규화, 텐서변환 등
        #coco dataset load를 위해 self.set_name작성해줌->추후 불러올때 train,val 분할 가능
        self.coco=COCO(os.path.join(self.data_dir+'/'+'annotations'+'/'+'instances_'+self.set_name+'.json')) #라벨링 파일로 cocodataset load
        #이미지 고유 아이디, class 저장
        self.image_ids=self.coco.getImgIds() #이미지 전체 인덱스를 저장 0,,,,,N까지의 인덱스 저장
        #self.imgInfo=self.coco.loadImgs(self.image_ids)
        self.load_classes() #self.classes라는 인스턴스 변수를 생성, coco데이터셋에서 사용된 클래스의 이름을 이 변수에 저장함, 딕셔너리를 출력하면 데이터셋에서 사용된
        # 모든 클래스 이름 확인 가능
        
    def load_classes(self):
        #Load class names(name->label)
        categories = self.coco.loadCats(self.coco.getCatIds()) #클래스별 고유 아이디, name, supercategory값 저장,[{'id': 0, 'name': 'disease', 'supercategory': 'none'},
        categories.sort(key=lambda x: x['id']) #id순 정리
        self.classes={} #빈 딕셔너리 생성
        self.coco_labels={} #라벨값을 담을 빈 딕셔너리 생성
        self.coco_labels_inverse={} #라벨값 내림차순 정렬 빈 딕셔너리 생성인듯 밑에
        
        for c in categories:
            self.coco_labels[len(self.classes)]=c['id'] #self.coco_labels에 0~label마지막 id값 저장,  {0:0,1:1}.....
            self.coco_labels_inverse[c['id']]=len(self.classes) #labels와 같음, ex) id=0->len=0, id=1->len=1
            self.classes[c['name']]=len(self.classes) #disease:13......
        self.labels={}
        
        for key,value in self.classes.items():
            self.labels[value]=key #13:disease....
            
    def __len__(self):
        return len(self.image_ids) #전체 이미지 장수개수 만큼 리턴
    
    def __getitem__(self,idx):
        img=self.load_image(idx) #load_image함수는 밑에 있음
        annot=self.load_annotations(idx) #load_annotations함수는 밑에 있음
        sample={'img':img,'annot':annot} #이미지 배열값, x,y,x1,x2값
        #print(sample)
        if self.transform:
            sample=self.transform(sample)
        
        return sample
    
    def load_image(self,image_index): #image_index를 넣어주면 실행됨
        #img=None
        image_info=self.coco.loadImgs(self.image_ids[image_index])[0] #id, filename,width,height 추출
        path=os.path.join(self.data_dir,'images',image_info['file_name'])
        img=skimage.io.imread(path)
        
        #클래스 폴더가 나누어진 경우 아래의 코드 주석 풀고 실행
        # try:
        #     if 'LA_NA' in image_info['file_name']:
        #         path=os.path.join(self.data_dir,'images',classes[0],image_info['file_name'])
        #         img=skimage.io.imread(path) #이미지 numpy변환
        #     elif 'AA_NA' in image_info['file_name']:
        #         path=os.path.join(self.data_dir,'images',classes[1],image_info['file_name'])
        #         img=skimage.io.imread(path) #이미지 numpy변환
        #     else:
        #         raise FileNotFoundError('Image file not found.')
        # except FileNotFoundError:
        #     raise FileNotFoundError(f"Image file not found for image index {image_index}.")
                
        if len(img.shape)==2: #흑백 이미지 (480,480)이런식
            img=skimage.color.gray2rgb(img) #흑백 이미지 컬러이미지 변환
            
        return img.astype(np.float32)/255 #이미지 배열값
    
    def load_annotations(self,image_index):
        annotations_ids=self.coco.getAnnIds(self.image_ids[image_index],iscrowd=False) #iscrowd=False->각각의 객체를 인식하기 위해, 이미지안에 객체 인덱스
        annotations=np.zeros((0,5)) #class,x,y,width,height 넣어주기 위함 빈 배열임
        
        if len(annotations_ids)==0: #객체가 없으면 함수 실행 종료
            return annotations
        
        coco_annotations=self.coco.loadAnns(annotations_ids) #id, image_id.category_id, bbox........
        for idx, a in enumerate(coco_annotations): #idx=index, a=id,image_id,category_id,bbox
            if a['bbox'][2] < 1 or a['bbox'][3] <1: #width, height가 no width or no height
                continue
                
            annotation=np.zeros((1,5)) #[0,0,0,0,0]
            annotation[0,:4]=a['bbox'] #1,2,3,4번째에 각각 x,y,width,height
            annotation[0,4]=self.coco_label_to_label(a['category_id']) #마지막에는 class고유값
            annotations=np.append(annotations,annotation,axis=0) #배열 append
            
        # transform from [x,y,w,h] to [x1,y1, x2, y2]
        annotations[:,2]=annotations[:,0]+annotations[:,2] #x2=x+width
        annotations[:,3]=annotations[:,1]+annotations[:,3] #y2=y+height
        
        return annotations #[x1,y1,x2,y2,category_id]
    
    def coco_label_to_label(self,coco_label):
        return self.coco_labels_inverse[coco_label]
    
    def label_to_coco_label(self,label):
        return self.coco_labels[label]
        
    def image_aspect_ratio(self,image_index):
        image=self.coco.loadImgs(self.image_ids[image_index])[0] #dictionary 형태id' {0,'license': 1,'file_name': '01_1_R_AB_LI_20220715_06_0204_jpg.rf.8860e7904cb6a8f0a601e32d63ad3e24.jpg','height': 480,'width': 480,
        return float(image['width']) / float(image['height'])
        
        
    def num_classes(self):
        return len(self.classes)
    
    def class_list_final(self):
        return self.classes

In [4]:
class Coco_testdata(Dataset):
    def __init__(self,data_dir,set_name,transform=None): 
        self.data_dir=data_dir #train,valid, test까지의 경로,O
        self.set_name=set_name #train2017, val2017->train까지의 경로일 시, test경로일시->test2017,O
        self.transform=transform #이미지 정규화, 텐서변환 등
        #coco dataset load를 위해 self.set_name작성해줌->추후 불러올때 train,val 분할 가능
        self.coco=COCO(os.path.join(self.data_dir+'/'+'annotations'+'/'+'instances_'+self.set_name+'.json')) #라벨링 파일로 cocodataset load
        #이미지 고유 아이디, class 저장
        self.image_ids=self.coco.getImgIds() #이미지 전체 인덱스를 저장 0,,,,,N까지의 인덱스 저장
        #self.imgInfo=self.coco.loadImgs(self.image_ids)
        self.load_classes() #self.classes라는 인스턴스 변수를 생성, coco데이터셋에서 사용된 클래스의 이름을 이 변수에 저장함, 딕셔너리를 출력하면 데이터셋에서 사용된
        # 모든 클래스 이름 확인 가능
        
    def load_classes(self):
        #Load class names(name->label)
        categories = self.coco.loadCats(self.coco.getCatIds()) #클래스별 고유 아이디, name, supercategory값 저장,[{'id': 0, 'name': 'disease', 'supercategory': 'none'},
        categories.sort(key=lambda x: x['id']) #id순 정리
        self.classes={} #빈 딕셔너리 생성
        self.coco_labels={} #라벨값을 담을 빈 딕셔너리 생성
        self.coco_labels_inverse={} #라벨값 내림차순 정렬 빈 딕셔너리 생성인듯 밑에
        
        for c in categories:
            self.coco_labels[len(self.classes)]=c['id'] #self.coco_labels에 0~label마지막 id값 저장,  {0:0,1:1}.....
            self.coco_labels_inverse[c['id']]=len(self.classes) #labels와 같음, ex) id=0->len=0, id=1->len=1
            self.classes[c['name']]=len(self.classes) #disease:13......
        self.labels={}
        
        for key,value in self.classes.items():
            self.labels[value]=key #13:disease....
            
    def __len__(self):
        return len(self.image_ids) #전체 이미지 장수개수 만큼 리턴
    
    def __getitem__(self,idx):
        img=self.load_image(idx) #load_image함수는 밑에 있음
        annot=self.load_annotations(idx) #load_annotations함수는 밑에 있음
        img_name=self.load_image_idx(idx)
        img_name={'filename':img_name}
        sample={'img':img,'annot':annot} #이미지 배열값, x,y,x1,x2값
        #print(sample)
        if self.transform:
            sample=self.transform(sample)
        
        return sample, img_name
    
    def load_image_idx(self,image_index):
        image_ids_test=self.coco.loadImgs(image_index)[0]['file_name']
        return image_ids_test
    
    def load_image(self,image_index): #image_index를 넣어주면 실행됨
        #img=None
        image_info=self.coco.loadImgs(self.image_ids[image_index])[0] #id, filename,width,height 추출
        path=os.path.join(self.data_dir,'images',image_info['file_name'])
        img=skimage.io.imread(path)
        # try:
        #     if 'LA_NA' in image_info['file_name']:
        #         path=os.path.join(self.data_dir,'images',classes[0],image_info['file_name'])
        #         img=skimage.io.imread(path) #이미지 numpy변환
        #     elif 'AA_NA' in image_info['file_name']:
        #         path=os.path.join(self.data_dir,'images',classes[1],image_info['file_name'])
        #         img=skimage.io.imread(path) #이미지 numpy변환
        #     else:
        #         raise FileNotFoundError('Image file not found.')
        # except FileNotFoundError:
        #     raise FileNotFoundError(f"Image file not found for image index {image_index}.")
        
        if len(img.shape)==2: #흑백 이미지 (480,480)이런식
            img=skimage.color.gray2rgb(img) #흑백 이미지 컬러이미지 변환
            
        return img.astype(np.float32)/255 #이미지 배열값
    
    def load_annotations(self,image_index):
        annotations_ids=self.coco.getAnnIds(self.image_ids[image_index],iscrowd=False) #iscrowd=False->각각의 객체를 인식하기 위해, 이미지안에 객체 인덱스
        annotations=np.zeros((0,5)) #class,x,y,width,height 넣어주기 위함 빈 배열임
        
        if len(annotations_ids)==0: #객체가 없으면 함수 실행 종료
            return annotations
        
        coco_annotations=self.coco.loadAnns(annotations_ids) #id, image_id.category_id, bbox........
        for idx, a in enumerate(coco_annotations): #idx=index, a=id,image_id,category_id,bbox
            if a['bbox'][2] < 1 or a['bbox'][3] <1: #width, height가 no width or no height
                continue
                
            annotation=np.zeros((1,5)) #[0,0,0,0,0]
            annotation[0,:4]=a['bbox'] #1,2,3,4번째에 각각 x,y,width,height
            annotation[0,4]=self.coco_label_to_label(a['category_id']) #마지막에는 class고유값
            annotations=np.append(annotations,annotation,axis=0) #배열 append
            
        # transform from [x,y,w,h] to [x1,y1, x2, y2]
        annotations[:,2]=annotations[:,0]+annotations[:,2] #x2=x+width
        annotations[:,3]=annotations[:,1]+annotations[:,3] #y2=y+height
        
        return annotations #[x1,y1,x2,y2,category_id]
    
    def coco_label_to_label(self,coco_label):
        return self.coco_labels_inverse[coco_label]
    
    def label_to_coco_label(self,label):
        return self.coco_labels[label]
        
    def image_aspect_ratio(self,image_index):
        image=self.coco.loadImgs(self.image_ids[image_index])[0] #dictionary 형태id' {0,'license': 1,'file_name': '01_1_R_AB_LI_20220715_06_0204_jpg.rf.8860e7904cb6a8f0a601e32d63ad3e24.jpg','height': 480,'width': 480,
        return float(image['width']) / float(image['height'])
        
        
    def num_classes(self):
        return len(self.classes)
    
    def class_list_final(self):
        return self.classes

In [ ]:
def collater_test(data):
    # img=sample['img'], annots=sample['annots'], scales=sample['scale']
    imgs=[s[0]['img'] for s in data] #이미지
    annots=[s[0]['annot'] for s in data] #annoation
    filename=[z['filename'] for s,z in data]
    scales=[s[0]['scale'] for s in data] #scale
    widths = [int(s.shape[0]) for s in imgs] #이미지 넓이
    heights = [int(s.shape[1]) for s in imgs] #이미지 높이
    batch_size = len(imgs) #배치사이즈

    max_width = np.array(widths).max() #max 이미지 넓이
    max_height = np.array(heights).max() #max 이미지 높이

    padded_imgs = torch.zeros(batch_size, max_width, max_height, 3) #max에 맞추어 0으로 패딩한 torch Tensor

    for i in range(batch_size):#batch당 이미지와 패딩Tensor 이미지 사이즈에 맞게 값을 넣어줌
        img = imgs[i]
        padded_imgs[i, :int(img.shape[0]), :int(img.shape[1]), :] = img

    max_num_annots = max(annot.shape[0] for annot in annots)
    
    if max_num_annots > 0:

        annot_padded = torch.ones((len(annots), max_num_annots, 5)) * -1

        if max_num_annots > 0:
            for idx, annot in enumerate(annots):
                #print(annot.shape)
                if annot.shape[0] > 0:
                    annot_padded[idx, :annot.shape[0], :] = annot
    else:
        annot_padded = torch.ones((len(annots), 1, 5)) * -1


    padded_imgs = padded_imgs.permute(0, 3, 1, 2)

    return {'img': padded_imgs, 'annot': annot_padded, 'scale': scales, 'filename':filename}

In [2]:
def collater(data):
    # img=sample['img'], annots=sample['annots'], scales=sample['scale']
    imgs=[s['img'] for s in data] #이미지
    annots=[s['annot'] for s in data] #annoation
    #filename=[s['filename'] for s in data]
    scales=[s['scale'] for s in data] #scale
    widths = [int(s.shape[0]) for s in imgs] #이미지 넓이
    heights = [int(s.shape[1]) for s in imgs] #이미지 높이
    batch_size = len(imgs) #배치사이즈

    max_width = np.array(widths).max() #max 이미지 넓이
    max_height = np.array(heights).max() #max 이미지 높이

    padded_imgs = torch.zeros(batch_size, max_width, max_height, 3) #max에 맞추어 0으로 패딩한 torch Tensor

    for i in range(batch_size):#batch당 이미지와 패딩Tensor 이미지 사이즈에 맞게 값을 넣어줌
        img = imgs[i]
        padded_imgs[i, :int(img.shape[0]), :int(img.shape[1]), :] = img

    max_num_annots = max(annot.shape[0] for annot in annots)
    
    if max_num_annots > 0:

        annot_padded = torch.ones((len(annots), max_num_annots, 5)) * -1

        if max_num_annots > 0:
            for idx, annot in enumerate(annots):
                #print(annot.shape)
                if annot.shape[0] > 0:
                    annot_padded[idx, :annot.shape[0], :] = annot
    else:
        annot_padded = torch.ones((len(annots), 1, 5)) * -1


    padded_imgs = padded_imgs.permute(0, 3, 1, 2)

    return {'img': padded_imgs, 'annot': annot_padded, 'scale': scales}

In [3]:
#image resize부분
class Resizer(object): #ndarrays->Tensor
    def __call__(self,sample,min_side=480,max_side=640): #min side, max side는 해당값으로 고정시켜주기
        image, annots =sample['img'], sample['annot'] #Cocodataset함수를 실행하며 sample변수가 생성됨 그것을 사용
        rows, cols, cns=image.shape #width, height, channels
        smallest_side=min(rows,cols) #너비, 높이 최소값
        # rescale the image->min_side/smallest_side->이미지를 최소한의 크기로 조정하기 위함
        scale=min_side/smallest_side #608/실제 가장 작은 값->이를 통해 이미지와 바운딩박스를 적절한 비율로 조정함
        #실제 max_side값 구하기
        largest_side=max(rows,cols) #가장 큰 너비, 혹은 높이
        
        if largest_side * scale > max_side: #max side보다 클 경우
            scale=max_side/largest_side #scale값 조정
            
        #resize the image with the computed scale-> scale값을 사용하여 이미지 Resize
        image=skimage.transform.resize(image, (int(round(rows*scale)), int(round((cols*scale)))))
        rows, cols, cns=image.shape #Resize한 row, col, cns
        #1024 side는 32x32 사이즈임->32행 32열중 어떠한 열을 padding할지 정하기 위해 32-rows%32하여 padding사이즈 구함
        pad_w=32-rows%32 #만약 489라면 9개의 행에 0을 추가하는 패딩
        pad_h=32-cols%32
        
        new_image=np.zeros((rows+pad_w,cols+pad_h,cns)).astype(np.float32)
        new_image[:rows,:cols,:]=image.astype(np.float32)
        
        annots[:,:4]*=scale #scale값 곱해주기
        return {'img':torch.from_numpy(new_image),'annot':torch.from_numpy(annots),'scale':scale}# from_numpy->numpy배열 Tensor로 변환

In [4]:
class Augmenter(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample, flip_x=0.5):

        if np.random.rand() < flip_x:
            image, annots = sample['img'], sample['annot']
            image = image[:, ::-1, :]

            rows, cols, channels = image.shape

            x1 = annots[:, 0].copy()
            x2 = annots[:, 2].copy()
            
            x_tmp = x1.copy()

            annots[:, 0] = cols - x2
            annots[:, 2] = cols - x_tmp

            sample = {'img': image, 'annot': annots}

        return sample

In [5]:
class Normalizer(object):

    def __init__(self):
        self.mean = np.array([[[0.485, 0.456, 0.406]]])
        self.std = np.array([[[0.229, 0.224, 0.225]]])

    def __call__(self, sample):

        image, annots = sample['img'], sample['annot']

        return {'img':((image.astype(np.float32)-self.mean)/self.std), 'annot': annots}

class UnNormalizer(object):
    def __init__(self, mean=None, std=None):
        if mean == None:
            self.mean = [0.485, 0.456, 0.406]
        else:
            self.mean = mean
        if std == None:
            self.std = [0.229, 0.224, 0.225]
        else:
            self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

In [6]:

class AspectRatioBasedSampler(Sampler):

    def __init__(self, data_source, batch_size, drop_last):
        self.data_source = data_source
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.groups = self.group_images()

    def __iter__(self):
        random.shuffle(self.groups)
        for group in self.groups:
            yield group

    def __len__(self):
        if self.drop_last:
            return len(self.data_source) // self.batch_size
        else:
            return (len(self.data_source) + self.batch_size - 1) // self.batch_size

    def group_images(self):
        # determine the order of the images
        order = list(range(len(self.data_source)))
        order.sort(key=lambda x: self.data_source.image_aspect_ratio(x))

        # divide into groups, one group = one batch
        return [[order[x % len(order)] for x in range(i, i + self.batch_size)] for i in range(0, len(order), self.batch_size)]